<a href="https://colab.research.google.com/github/4mami/SummarizeForEachRow/blob/main/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer, set_seed
from glob import glob
import torch
import tqdm

set_seed(1234)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

model = AutoModelWithLMHead.from_pretrained("t5-large")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("t5-large")

In [ ]:
# 1行が大体350 wordsぐらいのテキストファイルを読み込ませる
txt_files = glob('*.txt')
print(f"file name: {txt_files[0]}")

# テキストファイルを1行ずつ読んでいく
with open(txt_files[0]) as f:
  for line in tqdm.tqdm(f):
    line = line.rstrip()
    # その1行をトークナイズして、inputsを得る
    inputs = tokenizer.encode("summarize: " + line, return_tensors="pt")
    inputs = inputs.to(device)

    # inputsの長さを取得
    length = len(inputs[0])
    max_length = round(length * 0.2)
    min_length = round(length * 0.1)
    print(f"max_length: {max_length}, min_length: {min_length}")
    
    # 上の長さの10~20%になるようにoutputsを取得
    outputs = model.generate(inputs, max_length=max_length, min_length=min_length, num_beams=4, early_stopping=True)
    s = tokenizer.decode(outputs[0])
    
    # 出力用ファイルを開く
    with open("output.txt", "a") as output_f:
      # 出力用ファイルに1行分書き込む
      output_f.write(f"{s}\n")